# 弱学習器を使ったバンディットアルゴリズム

---

まず，今回はバンディットのアリゴリズムにUCBを使います．

UCBの詳しい解説はこちらの[ノートブック](https://github.com/yu-ki3406/Syumi-note/blob/main/notebooks/UCB.ipynb)に書いてあります．

まず簡潔に，UCBの定義をすると，次のような式になります．

$$
\bar{u}_i(t)=\hat{u}_i(t)+\sqrt{\frac{\log t}{N_i(t)}}
$$

$t$ は今までに試行した回数で，$N_i(t)$はt回までにiが引かれた回数,$\hat{u}_i(t)$はt回までのアームiの平均価値です．

右の式で1番大きくなったアームiを引いていくという流れになっています．

---

今回の弱学習器を使ったアルゴリズムでは$\hat{u}_i(t)$をなんらかの学習器に置き換えるという作業をします．

弱学習器の簡潔な理解としては，ランダムなモデルよりも少しだけ精度が良くなるというもです．

今回のアルゴリズムで次の考えは使わないのですが，弱学習器はアンサンブルという概念を使えるので，たくさんの弱学習器を使うことで，良い精度が出せることがわかっています．

例としては，XGBoostやニューラルネットワークなどが挙げられます．



In [35]:
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

#今回はアームが4つの場合を想定します，
class UCB_bandit:
    def __init__(self):
        '''
        N -> armがそれぞれ選ばれた回数
        rewards -> armのそれぞれの総報酬
        '''
        self.p = [0.1, 0.3, 0.5, 0.8] #最適なアームのインデックスは3
        self.N = np.zeros(len(self.p))
        self.rewards = np.zeros(len(self.p))
        self.d = [0,1,2,3]
        self.weights = np.ones(len(self.p))/len(self.p)
        self.models = [None] * len(self.p)



    def choose_action(self,t: int):
        #1回も選ばれてないアームをなくす．
        if t in [0,1,2,3]:
            return t

        rewards = np.zeros(len(self.p))
        for i in range(len(self.p)):
            # reward = self.rewards[i]/self.N[i] + 0.01 * np.sqrt(np.log(t) / (2*self.N[i]))
            reward = self.models[i].predict(np.array(self.d[i]).reshape(-1,1)) + 0.01 * np.sqrt(np.log(t) / (2*self.N[i]))
            rewards[i] = reward
        max_index = np.where(rewards == rewards.max())
        index = np.random.choice(max_index[0])

    
        return index
            

    def reward(self,index: int):

        return np.random.binomial(1,self.p[index],1)

    
    def update(self,index,reward):
        self.rewards[index] += reward
        self.N[index] += 1
        self.weights[index] = np.exp(1e-7*self.rewards[index]) * self.weights[index]
        X = self.d[index]
        y = self.rewards[index]/self.N[index]
        X,y = np.array(X).reshape(-1,1),np.array(y).reshape(-1,1)
        self.models[index] = self.weaker_learner(X,y,self.weights[index])
        


    def simulate(self,T):
        for i in tqdm(range(T)):
            arm = self.choose_action(i)
            
            reward = self.reward(arm)

            self.update(arm,reward)

    def weaker_learner(self,X,y,weights):
        model = LinearRegression()
        # print(X,y,weights)
        model.fit(X,y,sample_weight=weights)
        return model

In [36]:
agent = UCB_bandit()
agent.simulate(1000)
print(f'0が選ばれた回数:{agent.N[0]}回, 1が選ばれた回数:{agent.N[1]}回, 2が選ばれた回数:{agent.N[2]}回, 3が選ばれた回数:{agent.N[3]}回')

100%|██████████| 1000/1000 [00:00<00:00, 1170.04it/s]

0が選ばれた回数:1.0回, 1が選ばれた回数:9.0回, 2が選ばれた回数:1.0回, 3が選ばれた回数:989.0回
